In [2]:
import logging
from collections import defaultdict
import operator
import random

import pandas as pd
import numpy as np
from gensim import models, corpora, similarities
from sklearn import svm
from sklearn import linear_model
from sklearn.externals import joblib

import tensorflow as tf

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# data = pd.read_csv("tiantian_news_cutforsearch.csv")
data = pd.read_csv("raw_data/tiantian_news_cut.csv")

In [4]:
data = data.drop(data.loc[data["cutwords"].isnull()].index)

In [5]:
def get_categories():
    input_file = "conf/categories.txt"
    with open(input_file, "rb") as f:
        result = f.readlines()
        result = [x.strip('\n').decode('utf8') for x in result]
    return result

In [6]:
categories = get_categories()

In [7]:
sample_num = 5000
train = pd.DataFrame()
test = pd.DataFrame()

for cat in categories:
    data_onecat = data[data["category"] == cat.encode("utf8")]
    train_onecat = data_onecat.sample(n=sample_num)
    test_onecat = data_onecat.drop(train_onecat.index)
    train = pd.concat([train, train_onecat])
    test = pd.concat([test, test_onecat])
# test = test.sample(n=10000)
print train.shape
print test.shape

(100000, 4)
(422067, 4)


In [8]:
train_documents = train["cutwords"].tolist()
train_texts = [[word for word in document.split()] for document in train_documents]

In [9]:
test_documents = test["cutwords"].tolist()
test_texts = [[word for word in document.split()] for document in test_documents]

In [10]:
dictionary = corpora.Dictionary(train_texts)

2017-08-03 14:12:07,884 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


2017-08-03 14:12:12,961 : INFO : adding document #10000 to Dictionary(142380 unique tokens: [u'\u4e0d\u5b66\u65e0\u672f', u'\u4e66\u6bb5', u'\u5c97\u4e0a', u'\u6d2a\u6b66', u'\u51fb\u4e2d\u76ee\u6807']...)


2017-08-03 14:12:20,096 : INFO : adding document #20000 to Dictionary(248997 unique tokens: [u'\u535a\u878d', u'\u5e94\u5b66', u'\u4e66\u6bb5', u'\u94c5\u950c', u'\u5c97\u4e0a']...)


2017-08-03 14:12:27,756 : INFO : adding document #30000 to Dictionary(336134 unique tokens: [u'\u535a\u878d', u'TouchSense', u'\u5e94\u5b66', u'\u4e66\u6bb5', u'\u94c5\u950c']...)


2017-08-03 14:12:33,620 : INFO : adding document #40000 to Dictionary(405378 unique tokens: [u'\u85e4\u6912\u6cb9', u'\u535a\u878d', u'\u5357\u53ec', u'\u4e66\u6bb5', u'GF45mmf2']...)


...

2017-08-03 14:12:51,115 : INFO : adding document #60000 to Dictionary(555830 unique tokens: [u'\u85e4\u6912\u6cb9', u'\u535a\u878d', u'\u5357\u53ec', u'\u4e66\u6bb5', u'\u5f20\u7fce']...)


2017-08-03 14:12:57,183 : INFO : adding document #70000 to Dictionary(598305 unique tokens: [u'\u85e4\u6912\u6cb9', u'\u535a\u878d', u'\u5357\u53ec', u'\u4e66\u6bb5', u'360.00']...)


2017-08-03 14:13:06,126 : INFO : adding document #80000 to Dictionary(707158 unique tokens: [u'\u85e4\u6912\u6cb9', u'\u5434\u6d69\u5eb7', u'\u4e66\u6bb5', u'RogerWicker', u'\u8003\u5bdf\u961f\u5458']...)


2017-08-03 14:13:09,247 : INFO : adding document #90000 to Dictionary(724640 unique tokens: [u'\u85e4\u6912\u6cb9', u'\u5434\u6d69\u5eb7', u'\u4e66\u6bb5', u'RogerWicker', u'\u8003\u5bdf\u961f\u5458']...)


2017-08-03 14:13:16,836 : INFO : built Dictionary(767562 unique tokens: [u'\u85e4\u6912\u6cb9', u'\u6e34\u559c', u'\u5434\u6d69\u5eb7', u'\u4e66\u6bb5', u'RogerWicker']...) from 100000 documents (total 31533597 corpus positions)


In [11]:
train_corpus = [dictionary.doc2bow(text) for text in train_texts]

In [12]:
test_corpus = [dictionary.doc2bow(text) for text in test_texts]

In [13]:
tfidf = models.TfidfModel(train_corpus)

2017-08-03 14:21:04,727 : INFO : collecting document frequencies


2017-08-03 14:21:04,758 : INFO : PROGRESS: processing document #0


2017-08-03 14:21:05,824 : INFO : PROGRESS: processing document #10000


2017-08-03 14:21:06,817 : INFO : PROGRESS: processing document #20000


2017-08-03 14:21:07,913 : INFO : PROGRESS: processing document #30000


...

2017-08-03 14:21:11,526 : INFO : PROGRESS: processing document #60000


2017-08-03 14:21:12,476 : INFO : PROGRESS: processing document #70000


2017-08-03 14:21:13,837 : INFO : PROGRESS: processing document #80000


2017-08-03 14:21:14,341 : INFO : PROGRESS: processing document #90000


2017-08-03 14:21:15,467 : INFO : calculating IDF weights for 100000 documents and 767561 features (20122715 matrix non-zeros)


In [14]:
train_corpus_tfidf = tfidf[train_corpus]

In [15]:
test_corpus_tfidf = tfidf[test_corpus]

In [16]:
num_topics = 500

In [17]:
lsi = models.LsiModel(train_corpus_tfidf, id2word=dictionary, num_topics=num_topics)

2017-08-03 14:21:16,547 : INFO : using serial LSI version on this node


2017-08-03 14:21:16,549 : INFO : updating model with new documents


2017-08-03 14:24:37,650 : INFO : preparing a new chunk of documents


2017-08-03 14:24:38,864 : INFO : using 100 extra samples and 2 power iterations


2017-08-03 14:24:38,866 : INFO : 1st phase: constructing (767562, 600) action matrix


...

2017-08-03 15:28:14,323 : INFO : topic #0(26.036): 0.274*"孩子" + 0.146*"宝宝" + 0.109*"中国" + 0.102*"男人" + 0.091*"年" + 0.089*"妈妈" + 0.087*"女人" + 0.084*"电影" + 0.083*"美国" + 0.082*"爱"


2017-08-03 15:28:14,384 : INFO : topic #1(20.091): -0.471*"孩子" + -0.273*"宝宝" + 0.140*"中国" + -0.136*"妈妈" + 0.124*"市场" + 0.106*"美国" + 0.101*"年" + 0.099*"房地产" + 0.097*"房价" + 0.097*"城市"


2017-08-03 15:28:14,441 : INFO : topic #2(17.516): -0.469*"孩子" + -0.364*"宝宝" + 0.229*"男人" + 0.180*"女人" + 0.169*"星座" + 0.164*"狗狗" + 0.156*"爱情" + 0.126*"感情" + -0.110*"妈妈" + 0.109*"爱"


2017-08-03 15:28:14,493 : INFO : topic #3(16.400): 0.673*"狗狗" + 0.336*"主人" + -0.191*"孩子" + -0.172*"男人" + 0.163*"狗" + 0.150*"宝宝" + -0.138*"女人" + 0.115*"喵" + -0.105*"星座" + 0.105*"猫咪"


2017-08-03 15:28:14,529 : INFO : topic #4(15.627): -0.284*"狗狗" + 0.280*"宝宝" + -0.221*"孩子" + -0.188*"房价" + -0.163*"房地产" + -0.152*"城市" + 0.141*"手机" + -0.133*"主人" + 0.116*"车型" + 0.108*"设计"


In [18]:
# index = similarities.MatrixSimilarity(lsi[train_corpus])

In [19]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [20]:
clean_data = pd.concat([train, test])
clean_data["category"] = clean_data["category"].astype("category")
clean_data["category_encoded"] = clean_data["category"].cat.codes

In [21]:
train = clean_data[:sample_num * len(categories)]
test = clean_data[sample_num * len(categories):]

In [22]:
print train.shape
print test.shape

(100000, 5)
(422067, 5)


In [23]:
# query = test["cutwords"].tolist()

In [24]:
# len(test)

In [25]:
# total_num = 0
# top1_correct_num = 0
# top3_correct_num = 0
# top5_correct_num = 0
# top10_correct_num = 0
# 
# for i in range(len(test)):
#     try:
#         query_bow = dictionary.doc2bow(query[i].split())
#         query_lsi = lsi[query_bow]
#         sims = index[query_lsi]
#         sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])
#         sort_sims = sort_sims[:100]
#         predict = defaultdict(int)
#         for sort_sim in sort_sims:
#             predict_cat = train.iloc[sort_sim[0]]["category"]
#             predict[predict_cat] += sort_sim[1]
#         sorted_predict = sorted(predict.items(), key=operator.itemgetter(1), reverse=True)
#         
#         true_cat = test.iloc[i]["category"]
#         predict_cats = [item[0] for item in sorted_predict]
#         if true_cat == predict_cats[0]:
#             top1_correct_num += 1
#         else:
#             with open("logs/bad_case_top1.log", "ab") as f:
#                 f.write("{},{},{}\n".format(test.iloc[i]["id"], " ".join(predict_cats), test.iloc[i]["category"]))
#         if true_cat in predict_cats[:3]:
#             top3_correct_num += 1
#         if true_cat in predict_cats[:5]:
#             top5_correct_num += 1
#         if true_cat in predict_cats[:10]:
#             top10_correct_num += 1
#         else:
#             with open("logs/bad_case_top10.log", "ab") as f:
#                 f.write("{},{},{}\n".format(test.iloc[i]["id"], " ".join(predict_cats), test.iloc[i]["category"]))
#         total_num += 1
#         if i % 10 == 0:
#             with open("logs/run.log", "ab") as f:
#                 f.write("{},{},{},{},{},{}\n".format(i, top1_correct_num, top3_correct_num, top5_correct_num, top10_correct_num, total_num))
#     except:
#         with open("logs/exception.log", "ab") as f:
#             f.write("{}\n".format(i))
#         continue
#     # if i == 100:
#     #     break
# print top1_correct_num
# print top3_correct_num
# print top5_correct_num
# print top10_correct_num
# print total_num
# print float(top1_correct_num) * 100 / total_num
# print float(top3_correct_num) * 100 / total_num
# print float(top5_correct_num) * 100 / total_num
# print float(top10_correct_num) * 100 / total_num

In [26]:
# bad_case = pd.read_csv("logs/bad_case_top1.log", names=["id", "predict_category", "true_category"])

In [27]:
# a = test["category"].value_counts()
# b = bad_case["true_category"].value_counts()
# for cat in categories:
#     cat = cat.encode("utf8")
#     print "{}: {}".format(cat, float(b[cat]) * 100 / a[cat])

In [28]:
train_lsi = lsi[train_corpus]

In [29]:
train_x = [[t2[1] for t2 in t1] for t1 in train_lsi]

In [30]:
train_y = train["category_encoded"].values.tolist()

In [31]:
test_lsi = lsi[test_corpus]

In [32]:
test_x = [[t2[1] for t2 in t1] for t1 in test_lsi]

In [33]:
test_y = test["category_encoded"].values.tolist()

In [34]:
print len(train_x)
if len(train_x) != len(train_y):
    print "train x and y not match"
else:
    idx2del = []
    for i in range(len(train_x)):
        if not train_x[i] or (train_y[i] != 0 and not train_y[i]) or len(train_x[i]) != num_topics:
            idx2del.append(i)
    print idx2del
    for i in sorted(idx2del, reverse=True):
        del train_x[i]
        del train_y[i]
    print len(train_x)

100000
[]
100000


In [35]:
# clf = svm.SVC()
# clf.fit(train_x, train_y)

In [36]:
clf = linear_model.LogisticRegression(C=0.1, n_jobs=-1)
clf.fit(train_x, train_y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
# joblib.dump(clf, "svm_{}feas.pkl".format(num_topics))

In [38]:
print len(test_x)
if len(test_x) != len(test_y):
    print "test x and y not match"
else:
    idx2del = []
    for i in range(len(test_x)):
        if not test_x[i] or (test_y[i] != 0 and not test_y[i]) or len(test_x[i]) != num_topics:
            idx2del.append(i)
    print idx2del
    for i in sorted(idx2del, reverse=True):
        del test_x[i]
        del test_y[i]
    print len(test_x)

422067


[32805, 72842, 113020, 117655, 127501, 128237, 129809, 137487, 144854, 146245, 146668, 147357, 147906, 148524, 149253, 149561, 149811, 150512, 152915, 158545, 158548, 203952, 257723, 395217, 410622]
422042


In [39]:
correct_num = 0
for i in range(len(test_x)):
    predict = clf.predict([test_x[i]])
    if predict[0] == test_y[i]:
        correct_num += 1
    if i % 100 == 0:
        with open("logs/run.log", "ab") as f:
            f.write("{}, {}\n".format(i, correct_num))
print float(correct_num) * 100 / len(test_x)

87.1612303989


In [40]:
print float(correct_num) * 100 / len(test_x)

87.1612303989


In [41]:
correct_num = 0
for i in range(len(train_x)):
    predict = clf.predict([train_x[i]])
    if predict[0] == train_y[i]:
        correct_num += 1
    if i % 100 == 0:
        with open("logs/run.log", "ab") as f:
            f.write("{}, {}\n".format(i, correct_num))
print float(correct_num) * 100 / len(train_x)

88.096


In [42]:
print float(correct_num) * 100 / len(train_x)

88.096
